In [11]:
import pandas as pd
import json

In [12]:
DATA = 'website/src/data/'

df_scores = pd.read_csv(DATA + 'data_scores.csv', na_filter=False)

# drop columns
df_scores = df_scores.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

# rename columns
df_scores.columns = ['year', 'name', 'code', 'region', 'happiness']

df_scores

,year,name,code,region,happiness
0,2015,Switzerland,CH,Western Europe,7.587
1,2015,Iceland,IS,Western Europe,7.561
2,2015,Denmark,DK,Western Europe,7.527
3,2015,Norway,NO,Western Europe,7.522
4,2015,Canada,CA,North America,7.427
...,...,...,...,...,...
1360,2023,Congo (Kinshasa),CD,Sub-Saharan Africa,3.207
1361,2023,Zimbabwe,ZW,Sub-Saharan Africa,3.204
1362,2023,Sierra Leone,SL,Sub-Saharan Africa,3.138
1363,2023,Lebanon,LB,Middle East and Northern Africa,2.392


In [13]:
df_borders = pd.read_csv(DATA + 'country-borders.csv', na_filter=False)

# rename columns
df_borders.columns = ['code', 'name', 'border', 'border_name']

df_borders.drop(['name', 'border_name'], axis=1, inplace=True)

df_borders

,code,border
0,AD,FR
1,AD,ES
2,AE,OM
3,AE,SA
4,AF,CN
...,...,...
723,ZM,ZW
724,ZW,BW
725,ZW,MZ
726,ZW,ZA


In [14]:
df_gps = pd.read_csv(DATA + 'gps.csv', na_filter=False)

# rename columns
df_gps.columns = ['code', 'latitude', 'longitude', 'name']

df_gps = df_gps.drop(['name'], axis=1)
df_gps

,code,latitude,longitude
0,AD,42.546245,1.601554
1,AE,23.424076,53.847818
2,AF,33.93911,67.709953
3,AG,17.060816,-61.796428
4,AI,18.220554,-63.068615
...,...,...,...
240,YE,15.552727,48.516388
241,YT,-12.8275,45.166244
242,ZA,-30.559482,22.937506
243,ZM,-13.133897,27.849332


In [15]:
# merge df_scores and df_gps
df_nodes = pd.merge(df_scores, df_gps, on='code', how='inner')
df_nodes

,year,name,code,region,happiness,latitude,longitude
0,2015,Switzerland,CH,Western Europe,7.5870,46.818188,8.227512
1,2016,Switzerland,CH,Western Europe,7.5090,46.818188,8.227512
2,2017,Switzerland,CH,Western Europe,7.4940,46.818188,8.227512
3,2018,Switzerland,CH,Western Europe,7.4870,46.818188,8.227512
4,2019,Switzerland,CH,Western Europe,7.4800,46.818188,8.227512
...,...,...,...,...,...,...,...
1355,2021,Gambia,GM,Gambia,5.0510,13.443182,-15.310139
1356,2022,Gambia,GM,Gambia,5.1640,13.443182,-15.310139
1357,2023,Gambia,GM,Gambia,4.2790,13.443182,-15.310139
1358,2020,Maldives,MV,Maldives,5.1976,3.202778,73.22068


In [16]:
nodes = {}
codes = {}
id = 0

for index, row in df_nodes[["code", "name", "region", "latitude", "longitude"]].drop_duplicates().iterrows():
    nodes[row["code"]] = {
        'id': id,
        'code': row["code"],
        'name': row["name"],
        'region': row["region"],
        'latitude': row["latitude"],
        'longitude': row["longitude"],
        'happiness' : {},
    }

    codes[row["code"]] = id

    id += 1

for index, row in df_nodes.iterrows():
    nodes[row["code"]]["happiness"][row["year"]] = row["happiness"]

nodes

{'CH': {'id': 0,
  'code': 'CH',
  'name': 'Switzerland',
  'region': 'Western Europe',
  'latitude': '46.818188',
  'longitude': '8.227512',
  'happiness': {2015: 7.587,
   2016: 7.509,
   2017: 7.49399995803833,
   2018: 7.487,
   2019: 7.48,
   2020: 7.559899807,
   2021: 7.571,
   2022: 7.512,
   2023: 7.24}},
 'IS': {'id': 1,
  'code': 'IS',
  'name': 'Iceland',
  'region': 'Western Europe',
  'latitude': '64.963051',
  'longitude': '-19.020835',
  'happiness': {2015: 7.561,
   2016: 7.501,
   2017: 7.50400018692017,
   2018: 7.495,
   2019: 7.494,
   2020: 7.504499912,
   2021: 7.554,
   2022: 7.557,
   2023: 7.53}},
 'DK': {'id': 2,
  'code': 'DK',
  'name': 'Denmark',
  'region': 'Western Europe',
  'latitude': '56.26392',
  'longitude': '9.501785',
  'happiness': {2015: 7.527,
   2016: 7.526,
   2017: 7.52199983596802,
   2018: 7.555,
   2019: 7.6,
   2020: 7.645599842,
   2021: 7.62,
   2022: 7.636,
   2023: 7.586}},
 'NO': {'id': 3,
  'code': 'NO',
  'name': 'Norway',
  'reg

In [17]:
links = {}
# create links
for index, row in df_borders.iterrows():
    code = row["code"]
    border = row["border"]

    if code in codes and border in codes:
        if codes[code] not in links:
            links[codes[code]] = []

        links[codes[code]].append(codes[border])

links

{19: [21, 34],
 151: [83, 108, 80, 104, 69, 43],
 93: [100, 82, 91, 86],
 125: [79, 128, 108, 75],
 135: [137, 118, 160, 84],
 29: [50, 15, 26, 52, 31],
 12: [30, 25, 102, 49, 44, 54, 0],
 79: [125, 128, 108, 63, 75],
 94: [61, 82, 86],
 107: [115, 127],
 18: [28, 25, 16, 6],
 150: [153, 149, 112, 136, 142, 156],
 132: [100, 91, 85, 86, 75],
 155: [118, 152, 144],
 153: [150, 142, 77, 156],
 50: [29, 15, 26, 52, 57],
 15: [29, 50, 32, 52, 57, 39, 31, 22],
 78: [83, 115],
 126: [160, 111, 84, 113],
 58: [88, 55, 59, 63, 109],
 158: [42, 13],
 4: [14],
 118: [135, 155, 146, 137, 152, 144, 139, 84],
 146: [131, 147, 137, 118, 116],
 137: [135, 131, 146, 118, 141],
 0: [12, 28, 25, 49],
 149: [150, 112, 148, 114, 136],
 26: [29, 50, 57],
 131: [146, 147, 137, 141, 77],
 83: [151, 78, 71, 115, 53, 76, 97, 98, 127, 119, 80, 63, 104, 74],
 32: [15, 47, 24, 57, 22],
 11: [56, 24],
 30: [12, 25, 59, 44],
 25: [12, 18, 30, 2, 28, 16, 6, 59, 0],
 124: [120, 159],
 2: [25],
 96: [117],
 67: [62, 1

In [20]:
# create json
json_dict = {
    'nodes': [],
    'links': []
}


for code in nodes:
    json_dict['nodes'].append(nodes[code])

for code in links:
    for border in links[code]:
        json_dict['links'].append({
            'source': code,
            'target': border
        })

# Serializing json
json_object = json.dumps(json_dict, indent=4)
 
# Writing to sample.json
with open(DATA + "network.json", "w") as outfile:
    outfile.write(json_object)